In [47]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import os
import itertools
import cv2
from glob import glob
from PIL import Image
from sklearn.model_selection import train_test_split
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Flatten, Conv2D, MaxPool2D
from tensorflow.keras import backend as K
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau
from sklearn.metrics import classification_report

In [48]:
pwd

'/workspace/aman/MANYA'

In [49]:
df = pd.read_csv('GroundTruth.csv')
df.head()

,image,MEL,NV,BCC,AKIEC,BKL,DF,VASC
0,ISIC_0024306,0.0,1.0,0.0,0.0,0.0,0.0,0.0
1,ISIC_0024307,0.0,1.0,0.0,0.0,0.0,0.0,0.0
2,ISIC_0024308,0.0,1.0,0.0,0.0,0.0,0.0,0.0
3,ISIC_0024309,0.0,1.0,0.0,0.0,0.0,0.0,0.0
4,ISIC_0024310,1.0,0.0,0.0,0.0,0.0,0.0,0.0


In [50]:
df.shape

(10015, 8)

In [51]:
dfAKIEC = df[df["AKIEC"]==1]

In [52]:
dfAKIEC.head()

,image,MEL,NV,BCC,AKIEC,BKL,DF,VASC
23,ISIC_0024329,0.0,0.0,0.0,1.0,0.0,0.0,0.0
66,ISIC_0024372,0.0,0.0,0.0,1.0,0.0,0.0,0.0
112,ISIC_0024418,0.0,0.0,0.0,1.0,0.0,0.0,0.0
144,ISIC_0024450,0.0,0.0,0.0,1.0,0.0,0.0,0.0
157,ISIC_0024463,0.0,0.0,0.0,1.0,0.0,0.0,0.0


In [53]:
dfAKIEC.drop(columns=["MEL", "NV", "BCC", "BKL", "DF", "VASC"], inplace=True)

<ipython-input-53-40fa8da469b1>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dfAKIEC.drop(columns=["MEL", "NV", "BCC", "BKL", "DF", "VASC"], inplace=True)


In [54]:
dfAKIEC.head()

,image,AKIEC
23,ISIC_0024329,1.0
66,ISIC_0024372,1.0
112,ISIC_0024418,1.0
144,ISIC_0024450,1.0
157,ISIC_0024463,1.0


In [55]:
dfNotAKIEC = df[df["AKIEC"]==0]

In [56]:
dfNotAKIEC.tail()

,image,MEL,NV,BCC,AKIEC,BKL,DF,VASC
10010,ISIC_0034316,1.0,0.0,0.0,0.0,0.0,0.0,0.0
10011,ISIC_0034317,1.0,0.0,0.0,0.0,0.0,0.0,0.0
10012,ISIC_0034318,0.0,0.0,0.0,0.0,1.0,0.0,0.0
10013,ISIC_0034319,0.0,1.0,0.0,0.0,0.0,0.0,0.0
10014,ISIC_0034320,0.0,1.0,0.0,0.0,0.0,0.0,0.0


In [57]:
dfNotAKIEC.drop(columns=["MEL", "NV", "BCC", "BKL", "DF", "VASC"], inplace=True)

<ipython-input-57-13336ecb565e>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dfNotAKIEC.drop(columns=["MEL", "NV", "BCC", "BKL", "DF", "VASC"], inplace=True)


In [58]:
dfNotAKIEC.head()

,image,AKIEC
0,ISIC_0024306,0.0
1,ISIC_0024307,0.0
2,ISIC_0024308,0.0
3,ISIC_0024309,0.0
4,ISIC_0024310,0.0


In [59]:
dfNotAKIEC.shape

(9688, 2)

In [60]:
dfAKIEC.shape

(327, 2)

In [61]:
dfNotAKIEC = dfNotAKIEC.sample(frac = 0.04, random_state = 40)

In [62]:
dfAKIEC.shape

(327, 2)

In [63]:
dfNotAKIEC.shape

(388, 2)

In [64]:
df = pd.concat([dfAKIEC, dfNotAKIEC])

In [65]:
df.head()

,image,AKIEC
23,ISIC_0024329,1.0
66,ISIC_0024372,1.0
112,ISIC_0024418,1.0
144,ISIC_0024450,1.0
157,ISIC_0024463,1.0


In [66]:
df.shape

(715, 2)

In [67]:
df = df.sample(frac=1, random_state=30)

In [68]:
df.head()

,image,AKIEC
7432,ISIC_0031738,1.0
7746,ISIC_0032052,0.0
8065,ISIC_0032371,1.0
7391,ISIC_0031697,0.0
1906,ISIC_0026212,1.0


In [69]:
imageArray = df["image"].to_numpy()

In [70]:
imageArray.shape

(715,)

In [71]:
pwd

'/workspace/aman/MANYA'

In [72]:
cd images

/workspace/aman/MANYA/images


In [73]:
pwd

'/workspace/aman/MANYA/images'

In [74]:
Images = []

In [75]:
def processImage(image):
    image = cv2.resize(image, (150, 200))
    image = image.astype("float32") / 255
    return image

In [76]:
for img in imageArray:
    path = img + ".jpg"
    image = cv2.imread(path)
    image = processImage(image)
    Images.append(image)

In [77]:
len(Images)

715

In [78]:
cd ..

/workspace/aman/MANYA


In [79]:
np.save("AKEICIMAGES.npy", Images)

In [80]:
output = df["AKIEC"].to_numpy()

In [81]:
len(output)

715

In [82]:
output.shape

(715,)

In [83]:
xTrain, xTest, yTrain, yTest = train_test_split(np.array(Images), output, test_size = 0.2, random_state = 30)

In [84]:
xTrain.shape

(572, 200, 150, 3)

In [85]:
yTrain.shape

(572,)

In [86]:
xTrain, xVal, yTrain, yVal = train_test_split(xTrain, yTrain, test_size = 0.2, random_state = 30)

In [87]:
xVal.shape

(115, 200, 150, 3)

In [88]:
model = Sequential()
model.add(Conv2D(32, kernel_size=(3, 3),activation='relu',padding = 'Same',input_shape=(200, 150, 3)))
model.add(Conv2D(32,kernel_size=(3, 3), activation='relu',padding = 'Same',))
model.add(MaxPool2D(pool_size = (2, 2)))
model.add(Dropout(0.25))

model.add(Conv2D(64, (3, 3), activation='relu',padding = 'Same'))
model.add(Conv2D(64, (3, 3), activation='relu',padding = 'Same'))
model.add(MaxPool2D(pool_size=(2, 2)))
model.add(Dropout(0.40))

model.add(Flatten())
model.add(Dense(128, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(1, activation='sigmoid'))
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 200, 150, 32)      896       
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 200, 150, 32)      9248      
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 100, 75, 32)       0         
_________________________________________________________________
dropout (Dropout)            (None, 100, 75, 32)       0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 100, 75, 64)       18496     
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 100, 75, 64)       36928     
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 50, 37, 64)        0

In [89]:
model.compile(optimizer="adam", loss="binary_crossentropy", metrics=["Accuracy"])

In [90]:
history = model.fit(xTrain, yTrain, validation_data = (xVal, yVal), epochs = 10, batch_size = 32)

Epoch 1/10
15/15 [==============================] - 6s 221ms/step - loss: 5.7973 - accuracy: 0.0061 - val_loss: 0.6939 - val_accuracy: 0.0000e+00
Epoch 2/10
15/15 [==============================] - 1s 54ms/step - loss: 0.6926 - accuracy: 0.0000e+00 - val_loss: 0.7214 - val_accuracy: 0.0000e+00
Epoch 3/10
15/15 [==============================] - 1s 46ms/step - loss: 0.6919 - accuracy: 0.0000e+00 - val_loss: 0.6837 - val_accuracy: 0.0000e+00
Epoch 4/10
15/15 [==============================] - 1s 39ms/step - loss: 0.6761 - accuracy: 0.0000e+00 - val_loss: 0.6875 - val_accuracy: 0.0000e+00
Epoch 5/10
15/15 [==============================] - 0s 32ms/step - loss: 0.6709 - accuracy: 0.0000e+00 - val_loss: 0.6049 - val_accuracy: 0.0000e+00
Epoch 6/10
15/15 [==============================] - 0s 32ms/step - loss: 0.5919 - accuracy: 0.0000e+00 - val_loss: 0.6686 - val_accuracy: 0.0000e+00
Epoch 7/10
15/15 [==============================] - 0s 32ms/step - loss: 0.5948 - accuracy: 0.0000e+00 - val_

In [91]:
predictions = model.predict(xTest)

In [92]:
preds = []

In [93]:
for p in predictions:
    if p>=0.5:
        preds.append(1)
    else:
        preds.append(0)

In [94]:
report = classification_report(yTest, preds, target_names = ["NOT NV", "NV"])

In [95]:
print(report)

              precision    recall  f1-score   support

      NOT NV       0.95      0.49      0.65        73
          NV       0.65      0.97      0.78        70

    accuracy                           0.73       143
   macro avg       0.80      0.73      0.71       143
weighted avg       0.80      0.73      0.71       143

